In [1]:
import IPython

#import libraries
import sys
import tensorflow as tf
import numpy as np
import csv
from PIL import Image
import matplotlib.pyplot as plt

In [10]:
#Data Sets
BREED_TRAINING = "data/Train"
BREED_TRAINING_LABELS = "data/labels.csv"

BREED_TEST = "data/Test"
#testing_data = open("data/sample_submission.csv")

FEATURE_KEYS = ['image_matrix','breed']
#data type of input matrix is np.ndarray()

MODEL_PATH = "Models/"

#all data should already be downloaded

#parameters
train = True #start training NN
test = False #test NN
num_breeds = 120
num_steps = 1


image_matrix = tf.placeholder(tf.int8, shape =[250,250]) #shape = [None] ??

In [4]:
#get pixel data from image stored in folder
# example usage: 
#get_imgMatrix_from_id(data[h0][5])
def get_imgMatrix_from_id(image_id, image_dir="data/Train"):
    image_loc = image_dir + "/" + image_id + ".jpg"
    
    #return plt.imread(image_loc)
    image = tf.image.decode_jpeg(3) #Decode a JPEG-encoded image to a uint8 tensor
    resized_img = tf.image.resize_images(image,(250,250)) #resize all images to 250x250
    return resized_img

#new get_imgMatrix
#tf.image.deocde_jpeg 
#returns a unint8 tensor


In [1]:
feature_dict = { } #a dictionary in which the keys are feature names and the values are 
#Tensors (or SparseTensors) containing the corresponding feature data

headers = [] #arrays that holds all headers
labels = {"affenpinscher","afghan_hound","african_hunting_dog","airedale","american_staffordshire_terrier","appenzeller","australian_terrier","basenji","basset","beagle","bedlington_terrier","bernese_mountain_dog","black-and-tan_coonhound","blenheim_spaniel","bloodhound","bluetick","border_collie","border_terrier","borzoi","boston_bull","bouvier_des_flandres","boxer","brabancon_griffon","briard","brittany_spaniel","bull_mastiff","cairn","cardigan","chesapeake_bay_retriever","chihuahua","chow","clumber","cocker_spaniel","collie","curly-coated_retriever","dandie_dinmont","dhole","dingo","doberman","english_foxhound","english_setter","english_springer","entlebucher","eskimo_dog","flat-coated_retriever","french_bulldog","german_shepherd","german_short-haired_pointer","giant_schnauzer","golden_retriever","gordon_setter","great_dane","great_pyrenees","greater_swiss_mountain_dog","groenendael","ibizan_hound","irish_setter","irish_terrier","irish_water_spaniel","irish_wolfhound","italian_greyhound","japanese_spaniel","keeshond","kelpie","kerry_blue_terrier","komondor","kuvasz","labrador_retriever","lakeland_terrier","leonberg","lhasa","malamute","malinois","maltese_dog","mexican_hairless","miniature_pinscher","miniature_poodle","miniature_schnauzer","newfoundland","norfolk_terrier","norwegian_elkhound","norwich_terrier","old_english_sheepdog","otterhound","papillon","pekinese","pembroke","pomeranian","pug","redbone","rhodesian_ridgeback","rottweiler","saint_bernard","saluki","samoyed","schipperke","scotch_terrier","scottish_deerhound","sealyham_terrier","shetland_sheepdog","shih-tzu","siberian_husky","silky_terrier","soft-coated_wheaten_terrier","staffordshire_bullterrier","standard_poodle","standard_schnauzer","sussex_spaniel","tibetan_mastiff","tibetan_terrier","toy_poodle","toy_terrier","vizsla","walker_hound","weimaraner","welsh_springer_spaniel","west_highland_white_terrier","whippet","wire-haired_fox_terrier","yorkshire_terrier"}
#labels of what we are trying to predict

#input_fn doesn't need to handle ids, just handle image inputs
def input_fn(file_name, num_data, batch_size, is_training):
  """Creates an input_fn required by Estimator train/evaluate."""
    
    #Convert the inputs to a Dataset
    
    
    
    #preprocess your data here
    f = open(file_name, 'r',) #open csv file
    reader = csv.reader(f)
    
    headers = next(reader) #retrieve header names
    
    h0 = headers[0] #id
    h1 = headers[1] #breed
    h2 = "image" #uint8
    
    feature_dict[h0] = [] #list of ids
    feature_dict[h1] = [] #list of breeds
    feature_dict[h2] = [] #list of image arrays
    #data[headers[0]] = []
    # ...
        
    
    for row in reader: #loop through reader
        feature_dict[h0].append(row[0]) #appends id
        feature_dict[h1].append(row[1]) #appends breed
        feature_dict[h2].append(get_imgMatrix_from_id(row[0])) #appends image matrix based on id
        
    
    #...a tensor containing the labels
    #labels = the values your models aims to predict
       
    
    def _input_fn():
        """the input_fn"""
        dataset = tf.data.Dataset([filename])
        dataset = dataset.skip(1)
        dataset = dataset.map(_parse_csv)
        
        return features, labels
    
    return _input_fn

In [2]:
#define the input function -- see tensorflow documentation
#def input_fn =

In [3]:
#define a 3 layer neural network
def neural_net(x_dict): #pass in a dictionary of image matricies NOT IDs
    
    #3 layers 
    #TODO: Add more layers
    l0 = x_dict #the input layer
    l1 = tf.layers.dense(l0,256) #layer one, uses sigmoid function to calculate weights
    l2 = tf.layers.dense(l1, num_breeds) #the output layer
    
    return l2


In [4]:
#tf estimator model function
def model_fn(features, labels, mode, params):    
    #this function is where the most machine learning knowledge comes into play
    
    #build the neural network
    logits = neural_net(features)
    
    #Predictions
    pred_classes = tf.argmax(logits, axis=3)
    print("pred classes: " + pred_classes)
        
    #compute loss and optimizer - taken from Tensorflow custom_estimator doc
    loss_op = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits) #TODO:use signmoid instead
    train_op = optimizer.minimize(loss_op, )
            

    estimator = DNNEstimator(
        head=tf.contrib.estimator.multi_label_head(n_classes=3),
        feature_columns=[sparse_feature_a_emb, sparse_feature_b_emb],
        hidden_units=[1024, 512, 256])
    
    #specifies the required ops for the estimator and returns
    return tf.estimator.EstimatorSpec(
        mode=mode, #mode is not required, TODO:maybe createa a param for mode and access it here
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op)    
        


In [16]:
#2.0 define the feature columns
#exampple: image_ft = tf.feature_column.numeric_column('population')
#image_ft = tf.placeholder(tf.int8, shape =[250,250]) #shape = [None] ??

In [17]:
#build the estimator
#use tensorflow estimator -- see tensorflow documentation
#model = tf.estimator.Estimator.LinearClassifier(
#    feature_columns=[image_ft],
#    )

optimizer = tf.train.GradientDescentOptimizer(0.0001)



AttributeError: type object 'Estimator' has no attribute 'LinearClassifier'

In [11]:
#train the model
model.train(input_fn=input_fn, steps=num_steps)

NameError: name 'input_fn' is not defined

In [12]:
#test the network

ds_predict_tf = model.predict(get_imgMatrix_from_id("0a0b97441050bba8e733506de4655ea1",'data/Test'))
for i in ds_predict_tf:
    print(i)
                                                                                            

NameError: name 'model' is not defined

In [ ]:
#save the model

In [9]:
test = get_imgMatrix_from_id("0a0b97441050bba8e733506de4655ea1", image_dir="data/Test")
print(type(test))

<class 'numpy.ndarray'>
